In [11]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import time
import json
import urllib
import requests
from bs4 import BeautifulSoup
import random
import time
import warnings
warnings.filterwarnings(action='ignore')

In [12]:
df = pd.read_csv('./transactions.csv')
df

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at
0,970634,504760.0,201435.0,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,NaN,NaN,110500.0,110500.0,NaN,VBANK,개발자 커리어,NaN
1,970650,432862.0,203178.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,NaN,NaN,116000.0,116000.0,NaN,CARD,영상 편집,2020-12-02 00:00:00
2,970657,72297.0,204246.0,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090.0,190900.0,190900.0,NaN,CARD,NaN,2021-03-04 00:00:00
3,970656,72297.0,204246.0,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,NaN,NaN,190900.0,190900.0,NaN,CARD,NaN,2021-03-04 00:00:00
4,970658,478028.0,201797.0,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000.0,117500.0,117500.0,NaN,CARD,PPT/보고서,2019-11-14 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105414,1364995,640697.0,205365.0,PAYMENT,CANCELLED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:10:42,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,116000.0,INICIS,TRANS,영상 편집,2021-06-15 00:00:00
105415,1364998,670896.0,204159.0,PAYMENT,CANCELLED,부동산 디벨로퍼 아카데미 : 인허가,부동산/금융,올인원,2021-09-25 14:16:33,189000,NaN,NaN,189000.0,189000.0,INICIS,CARD,부동산 개발,2021-02-10 00:00:00
105416,1364996,640697.0,205365.0,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,116000.0,116000.0,INICIS,VBANK,영상 편집,2021-06-15 00:00:00
105417,1365005,670896.0,204159.0,PAYMENT,CANCELLED,부동산 디벨로퍼 아카데미 : 인허가,부동산/금융,올인원,2021-09-25 14:19:56,189000,NaN,NaN,189000.0,189000.0,INICIS,CARD,부동산 개발,2021-02-10 00:00:00


In [13]:
# 결제가는 NaN 값이 없으므로 쿠폰명을 모르지 할인액이나 쿠폰여부는 채울 수 있음
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 105419 entries, 0 to 105418
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   id                      105419 non-null  int64  
 1   customer_id             105416 non-null  float64
 2   course_id               100212 non-null  float64
 3   type                    105419 non-null  object 
 4   state                   105419 non-null  object 
 5   course_title            105403 non-null  object 
 6   category_title          105395 non-null  object 
 7   format                  105403 non-null  object 
 8   completed_at            105419 non-null  object 
 9   transaction_amount      105419 non-null  int64  
 10  coupon_title            31817 non-null   object 
 11  coupon_discount_amount  31817 non-null   float64
 12  sale_price              104291 non-null  float64
 13  tax_free_amount         104291 non-null  float64
 14  pg                  

# 삭제) customer_id  nan값 3건

In [14]:
df.dropna(subset=['customer_id'], inplace = True)   

In [173]:
df[df['customer_id'].isnull()]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,tax_free_amount,pg,method,subcategory_title,marketing_start_at


# 삭제) 중복 column 'Tex_free_'

In [15]:
df.drop(columns='tax_free_amount',inplace=True)

# 삭제) course_id.isnull & couse_title.isnull 13건
### 유추 가능하나 state DELETED 및 중복 매출 등으로 지표활용 어려움
- {'id' :[1093780, 1093784} = {course_id : 203657}
- {'id' : 1093777} = {course_id : 202545}
- {'id' : 1012747} = {course_id : 204013}
- {'id' : 1141801} = {course_id : 204013}

In [16]:
# 상품정보 확인 어려운 
trsh = df[(df['course_id'].isnull()) & (df['course_title'].isnull())].index
df.drop(trsh, inplace=True)

# 보류) 그외 course_id.isnull & couse_title.notnull 5,191건

### 'couse_title'.unique 309건
### 'course_title' 이 '프로모션 :'  또는 '[한정판매]' 로 시작
- course_id.isnull ⊅ 'course_title' 프로모션 = 'course_id' = [205796, 205807, 205925, 206065] 총593건

### 확인필요, int 값인데 ' ' 필요한가? object 되는거 아닌가

In [17]:
# 기존 'course_id' 와 중복안되게 30만번대로 진행(연월일)
df[(df['course_id'].isnull()) & (df['course_title'].notnull())].index
df.loc[df['course_id'] != df['course_id'], 'course_id'] = 300000

In [18]:
df[df['course_id'] == 300000] # 확인
df[df['course_id'].isnull()]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


### 위에서 지움_review 하면서 코드 삭제해도됨

In [38]:
# trsh2 = df[(df['course_id'].isnull()) & (df['course_title'].isnull())].index
# df.drop(trsh2, inplace=True)

Int64Index([], dtype='int64')

# 삭제) 이상 data : sale_price 및 transaction_amount  6건 이상

In [19]:
# 판매가, 결제가 1,000원 
# {'id' :[1118570} = {course_id : 204656}
trsh = df['id'].isin([1118570])
df = df[~trsh]

In [20]:
# 동일 cutomer_id 에 transaction -150,000원 만 있음
trsh = df['customer_id'].isin([482240])
df = df[~trsh]

In [21]:
# type REFUND 건으로 transaction -150,000원 만 있음
trsh = df['id'].isin([1078156])
df = df[~trsh]

In [22]:
# 판매가, 결제가 1,000원 
trsh = df['id'].isin([1302735])
df = df[~trsh]

In [23]:
# 판매가, 결제가 500원 
trsh = df['id'].isin([1008518])
df = df[~trsh]

In [24]:
# 판매가, 결제가 444,444원, B2B로 특정 및 유사 data 확인 안됨 
trsh = df['id'].isin([1094558])
df = df[~trsh] 

In [25]:
# 데이터가 잘들어있긴한데 애매함 ID : 1172079, 1189862, 1189870, 1189875, 1189900
trsh = df['customer_id'].isin([542837])
df = df[~trsh] 

### 확인필요 아래 ID

In [ ]:
# #  1304724 1337870 1337550 1259540
# trsh = df['id'].isin([1304724])
# df = df[~trsh]

### 확인필요 {'id' :1118481} sale_price 만 이상하고 transaction은 정상인데 지워야할지.. state CANCELLED

In [29]:
# # 판매가 1,000원, 결제가 229,000원
# # {'id' :[1118481} = {course_id : 204656}
# trsh = df['id'].isin([1118481])
# df = df[~trsh]
df[df['id'] == 1118481]    # 취소율 구할때는 적용하고 매출로 할때는 지우기!!  sale_price 수정하기

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
47761,1118481,147112.0,204656.0,ORDER,CANCELLED,경영의 신: core MBA,지식콘텐츠,MBA,2021-06-03 14:58:47,229000,NaN,NaN,229000.0,KAKAO,NaN,NaN,2021-06-04 00:00:00


In [28]:
df.loc[(df.sale_price== 1000), 'sale_price'] = 229000  
# 취소율 구할때는 적용하고 매출로 할때는 지우기!!  sale_price 수정하기

### 확인 필요, course_id 205579 구매후 9/8 203178 1+1프로모션 0원 구매, 9/9 203178 0원 환불. 미친놈인가...  그래도 'customer_id' 가 아니라 'id' 별 삭제로 1286762 는 남길만함
 'id' : [1286762, 1336569, 1337870] 
 'customer_id' : [632837, 632837, 632837]
 'course_id : [205579, 203178, 203178]

In [205]:
# trsh = df['customer_id'].isin([632837.0])
# df = df[~trsh]

In [35]:
df[df['customer_id'].isin([632837.0])]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


In [30]:
#  3개 날리기 
trsh = df['id'].isin([1286762])
df = df[~trsh]

In [32]:
#  3개 날리기 
trsh = df['id'].isin([1336569])
df = df[~trsh]

In [34]:
#  3개 날리기 
trsh = df['id'].isin([1337870])
df = df[~trsh]

### 확인 필요, 1개의 'course_title' 로 2개의 'customer_id' : [29356, 507572], 각 3건의 'course_id' 206840 할당. coupon cloumn이 누락됬지만 'id' 1264416 의 경우 'method' 프로모션으로 무료제공 가능성 있음

In [120]:
# # course_id 봐야함
# trsh = df['customer_id'].isin([29356.0])
# df = df[~trsh]

In [38]:
df[df['course_id'] == 206840]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


In [36]:
trsh = df['customer_id'].isin([632837.0])
df = df[~trsh]  # B2B날리기

In [37]:
trsh = df['course_id'].isin([206840])
df = df[~trsh]  # B2B날리기

# 수정) category_title NaN data 입력 7 건
- ['course_id'].isnull() & ['course_title'].notnull() & ['category_title'].isnull()

In [44]:
df[df['category_title'].isnull()]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


In [ ]:
# #인덱스로 한땀한땀 변경
# df.loc[105154, 'category_title'] = '영상/3D'
# df.loc[104284, 'category_title'] = '영상/3D'
# df.loc[104022, 'category_title'] = '영상/3D'
# df.loc[74959, 'category_title'] = '데이터사이언스'
# df.loc[74954, 'category_title'] = '데이터사이언스'
# df.loc[74952, 'category_title'] = '데이터사이언스'
# df.loc[72618, 'category_title'] = '데이터사이언스'

In [42]:
for i in (104056, 104318, 105189):
    df.loc[i, 'category_title'] = '영상/3D'

In [43]:
for i in (72639,74973, 74975,  74980):
    df.loc[i, 'category_title'] = '데이터사이언스'

In [190]:
# 확인해보면 잘바껴서 더이상 검색안댐_하나 밑에서 날릴예정
df[df['category_title'].isnull()]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


# 수정) 'pg' column NaN 값 'COMM' 입력 15,072 건

In [46]:
df.loc[df['pg'] != df['pg'], 'pg'] = 'COMM'
df # 확인
df['pg'].unique()

array(['COMM', 'INICIS', 'KAKAO', 'CHAI', 'PAYCO', 'NAVER'], dtype=object)

In [276]:
# 이거 왜했던거지?, 유니크 왜늘어났지?
df[df['method'].isnull()]['pg'].unique()

array([], dtype=object)

# 수정) 'method' TRANS == TRANSFER 1,247 건

In [45]:
df.loc[(df.method == 'TRANSFER'), 'method'] = 'TRANS'

In [47]:
df.loc[df['method'] != df['method'], 'method'] = 'N_TRANS'
df[df['method'] == 'N_TRANS']

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
21515,1035779,147114.0,203527.0,ORDER,CANCELLED,올인원 패키지 : 김민태의 프론트엔드 아카데미 : 제 1강 JavaScript & ...,프로그래밍,올인원,2021-04-28 14:01:25,99000,NaN,NaN,99000.0,KAKAO,N_TRANS,프론트엔드 개발,2021-04-07 00:00:00
21516,1035784,167967.0,203532.0,ORDER,CANCELLED,연습문제 패키지 : 퀴즈처럼 풀면서 배우는 파이썬 딥러닝 300제+,데이터사이언스,올인원,2021-04-28 14:01:56,92000,NaN,NaN,92000.0,CHAI,N_TRANS,딥러닝/인공지능,2021-02-22 00:00:00
21517,1035786,147114.0,203527.0,ORDER,CANCELLED,올인원 패키지 : 김민태의 프론트엔드 아카데미 : 제 1강 JavaScript & ...,프로그래밍,올인원,2021-04-28 14:02:27,99000,NaN,NaN,99000.0,PAYCO,N_TRANS,프론트엔드 개발,2021-04-07 00:00:00
21518,1035778,485705.0,203787.0,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 디자인 툴,디자인,올인원,2021-04-28 14:02:53,32000,4월 1+1 페이백 이벤트 쿠폰,140000.0,172000.0,KAKAO,N_TRANS,디자인툴,2020-12-24 11:00:00
21521,1035794,167967.0,203532.0,ORDER,CANCELLED,연습문제 패키지 : 퀴즈처럼 풀면서 배우는 파이썬 딥러닝 300제+,데이터사이언스,올인원,2021-04-28 14:03:32,92000,NaN,NaN,92000.0,PAYCO,N_TRANS,딥러닝/인공지능,2021-02-22 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105350,1364698,646968.0,205879.0,PAYMENT,CANCELLED,올인원 패키지 : 누적 다운로드 120만+ 1인 개발자와 함께하는 앱 개발 입문,프로그래밍,올인원,2021-09-25 02:29:26,149000,[WELCOME] 온라인 강의 3만원 할인쿠폰,30000.0,179000.0,NAVER,N_TRANS,모바일앱,2021-07-02 00:00:00
105351,1364699,646968.0,206888.0,PAYMENT,CANCELLED,올인원 패키지 : 글로벌 1위 Salesforce 디자인 디렉터의 B2B SaaS ...,디자인,올인원,2021-09-25 02:30:51,149000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000.0,169000.0,KAKAO,N_TRANS,UX/UI,2021-08-23 00:00:00
105390,1364858,37866.0,205707.0,PAYMENT,CANCELLED,올인원 패키지 : 15명의 전문 애널리스트에게 배우는 산업&매크로 분석과 투자 전략,투자/재테크,올인원,2021-09-25 11:17:16,339000,NaN,NaN,339000.0,KAKAO,N_TRANS,주식 투자,2021-06-23 00:00:00
105396,1364879,623392.0,206060.0,PAYMENT,CANCELLED,초격차 패키지 : 한 번에 끝내는 컴퓨터 공학 전공필수 & 인공지능 심화,프로그래밍,올인원,2021-09-25 11:38:53,173000,[WELCOME] 온라인 강의 3만원 할인쿠폰,30000.0,203000.0,KAKAO,N_TRANS,개발자 커리어,2021-07-16 00:00:00


In [48]:
df['method'].unique()

array(['VBANK', 'CARD', 'TRANS', 'PROMOTION', 'ONSITE', 'N_TRANS',
       'POINT', 'VOUCHER'], dtype=object)

# 수정) 'transaction' 대비 'coupon_title', 'coupon_discount_amount', 'sale_price' NaN 값 입력_ 근데 늦었으니까 그냥 어제처럼 가자 적용만 살리고



### ['transaction'].notnull() & ['sale_price'].isnull 인 것 1,108 건
- 주로 B2B 상품, 
- 'sale_price' = 'transaction'금액, 

### ['sale_price'] == 1 을 0으로 바꿔주고 1,192 건
- 스쿨, 스쿨올라인(무료제공강좌) : 이것도 쿠폰은 안적혀 있지만 위에 쿠폰 정책에 따라 날려야할까?


### ['transaction'] == 0 &  ['sale_price'].notnull & ['coupon_discount_amount'].isnull 인것 17 건
- 주로 올인원, B2B 상품
- 'coupon_discaount_amount' = 'sale_price'금액, 'coupon_title' = '적용'

### ['transaction'] 과 ['sale_price'] 의 각 row 값이 같고 &  ['coupon_discount_amount'].isnull 이면
- 'coupon_discaount_amount' = 'sale_price'금액, 'coupon_title' = '미적용'


In [57]:
coo = df[df['course_title'].str.contains('B2B')]
coo['course_title'].unique()

array(['(B2B) 현대백화점 온라인 구독 올 플랜', '(B2B) 힐코코리아 온라인 구독 올 플랜 - 패파입주사',
       '(B2B) 주식회사 루나 온라인 구독 ALL PLAN - 패파입주사', ...,
       '(B2B) 그린오션스 온라인 구독 올 플랜 - 비대면바우처',
       '(B2B) 디캠프 패밀리사 온라인 구독 올 플랜 (9월)',
       '(B2B) (주)오디 온라인 구독 올 플랜 - 비대면바우처'], dtype=object)

In [60]:
df.loc[(df.sale_price == 1.0), 'sale_price'] = 0    # sale_price : 1.0 인거 0 으로 바꾸기

In [216]:
a = df['coupon_title'].isnull().value_counts()
b = df['coupon_discount_amount'].isnull().value_counts()
c = df['sale_price'].isnull().value_counts()
a, b, c

(True     73580
 False    31817
 Name: coupon_title, dtype: int64,
 True     73580
 False    31817
 Name: coupon_discount_amount, dtype: int64,
 False    104289
 True       1108
 Name: sale_price, dtype: int64)

In [ ]:
# coupon_title 미적용
df.loc[df['coupon_title'] != df['coupon_title'], 'coupon_title'] = '미적용'
df # 확인
df[df['coupon_title'].isnull()]

In [88]:
df.isnull().sum()

id                            0
customer_id                   0
course_id                     0
type                          0
state                         0
course_title                  0
category_title                0
format                        0
completed_at                  0
transaction_amount            0
coupon_title              72209
coupon_discount_amount    72209
sale_price                 1062
pg                            0
method                        0
subcategory_title         18404
marketing_start_at        16554
dtype: int64

In [90]:
# coupon_title 적용

df[df['id'] == 1264416]
#df[df['id'] == 1337550]
#df[df['id'] == 1304724]
#df[df['id'] == 1259540]
# df[df['id'] == 1214209]
# df[df['id'] == 1214238]
# df[df['id'] == 1318662]
# df[df['id'] == 1284320]
# df[df['id'] == 1284322]
# df[df['id'] == 1129305]
# df[df['id'] == 1303022]
# df[df['id'] == 1118431]
# df[df['id'] == 977734]
# df[df['id'] == 1280563]
# df[df['id'] == 1337581]
# df[df['id'] == 977735]
# df[df['id'] == 1337870]

,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at


In [78]:
b2g = df['format'].isin(['B2G'])
df = df[~b2g]

In [120]:
b2b = df['format'].isin(['B2B'])
df = df[~b2b]

In [87]:
testt = df['coupon_title'].isin(['테스트'])
df = df[~testt]

In [113]:
df = df.drop([923787,100368], axis = 0)

In [135]:
sh = df['format'].isin(['B2B', 'B2B 온라인', 'B2G', '관리', '스쿨', '스쿨온라인', '캠프'])
df = df[~sh]

In [171]:
df.isnull().sum()

id                            0
customer_id                   0
course_id                     0
type                          0
state                         0
course_title                  0
category_title                0
format                        0
completed_at                  0
transaction_amount            0
coupon_title                  0
coupon_discount_amount        0
sale_price                   31
pg                            0
method                        0
subcategory_title         16279
marketing_start_at        15467
dtype: int64

In [106]:
#df.loc[, 'coupon_title'] = '적용'
#df.loc[100283, 'coupon_title'] = '적용'
#df.loc[923787, 'coupon_title'] = '적용'
#df.loc[83412, 'coupon_title'] = '적용'
#df.loc[72897, 'coupon_title'] = '적용'
#df.loc[72902, 'coupon_title'] = '적용'
#df.loc[95790, 'coupon_title'] = '적용'
#df.loc[88167, 'coupon_title'] = '적용'
#df.loc[88169, 'coupon_title'] = '적용'
#df.loc[51228, 'coupon_title'] = '적용'
#df.loc[92430, 'coupon_title'] = '적용'
#df.loc[47744, 'coupon_title'] = '적용'
#df.loc[2635, 'coupon_title'] = '적용'
#df.loc[87364, 'coupon_title'] = '적용'
#df.loc[100294, 'coupon_title'] = '적용'
#df.loc[2636, 'coupon_title'] = '적용'
#df.loc[100368, 'coupon_title'] = '적용'

In [ ]:
# # sale_price 금액이 없네..
# for i in (84094, 100283, 92787,83412, 72897,72902,95790,88167,88169,51228,92430,47744,2635,87364,100294,2636,100368):
#     df.loc[i, 'coupon_discount_amount'] = '영상/3D'

In [174]:
a = df[df['sale_price'].isnull()]
s = []
for i in a['transaction_amount']:
    L = df.loc[df['sale_price'] != df['sale_price'], 'sale_price'] = abs(i)
    s.append(L)
print(s)

[101000.0, 88000.0, 85000.0, 168100.0, 120500.0, 138000.0, 125500.0, 100000.0, 142500.0, 22500.0, 78400.0, 269100.0, 94500.0, 149000.0, 124500.0, 60223.0, 62958.0, 50032.0, 84700.0, 250000.0, 250000.0, 99000.0, 250000.0, 250000.0, 138000.0, 138000.0, 138000.0, 141000.0, 139000.0, 126000.0, 126000.0]


In [182]:
df1.isnull().sum()

id                            0
customer_id                   0
course_id                     0
type                          0
state                         0
course_title                  0
category_title                0
format                        0
completed_at                  0
transaction_amount            0
coupon_title                  0
coupon_discount_amount        0
sale_price                   31
pg                            0
method                        0
subcategory_title             0
marketing_start_at        15467
dtype: int64

In [181]:
a = df1[df1['subcategory_title'].isnull()]
s = []
for i in a['category_title']:
    L = df1.loc[df1['subcategory_title'] != df1['subcategory_title'], 'subcategory_title'] = i
    s.append(L)
print(s)

['프로그래밍', '프로그래밍', '크리에이티브', '부동산/금융', '부동산/금융', '프로그래밍', '프로그래밍', '프로그래밍', '크리에이티브', '데이터사이언스', '데이터사이언스', '프로그래밍', '데이터사이언스', '프로그래밍', '프로그래밍', '프로그래밍', '프로그래밍', '데이터사이언스', '데이터사이언스', '데이터사이언스', '프로그래밍', '마케팅', '프로그래밍', '프로그래밍', '데이터사이언스', '프로그래밍', '프로그래밍', '프로그래밍', '프로그래밍', '데이터사이언스', '프로그래밍', '데이터사이언스', '데이터사이언스', '프로그래밍', '크리에이티브', '프로그래밍', '마케팅', '프로그래밍', '프로그래밍', '프로그래밍', '부동산/금융', '크리에이티브', '크리에이티브', '데이터사이언스', '프로그래밍', '마케팅', '업무 생산성', '업무 생산성', '부동산/금융', '부동산/금융', '부동산/금융', '데이터사이언스', '프로그래밍', '프로그래밍', '데이터사이언스', '크리에이티브', '데이터사이언스', '프로그래밍', '프로그래밍', '데이터사이언스', '크리에이티브', '부동산/금융', '마케팅', '부동산/금융', '부동산/금융', '프로그래밍', '데이터사이언스', '데이터사이언스', '프로그래밍', '프로그래밍', '부동산/금융', '데이터사이언스', '프로그래밍', '프로그래밍', '크리에이티브', '프로그래밍', '프로그래밍', '프로그래밍', '프로그래밍', '데이터사이언스', '프로그래밍', '프로그래밍', '크리에이티브', '크리에이티브', '프로그래밍', '데이터사이언스', '데이터사이언스', '부동산/금융', '크리에이티브', '부동산/금융', '프로그래밍', '프로그래밍', '프로그래밍', '프로그래밍', '프로그래밍', '크리에이티브', '크리에이티브', '데이터사이언스', '업무 생산성', '업무 생산성', '마케팅', '업무 생산성', '데이터사이언스', '데이터사이

unique 가 왜 늘어났지?

# 보류) 구매결정 요인이 아니라고 보이는  'coupon_title' 732건, 195종
### 서포터즈, 강사용, 테스트, 직원검수 등 
### 문제는
- B2G 제공강좌 쿠폰은 어떻게 할꺼야? B2G도 선행 학습이나 이런거 보고 신청할 수도 있는데
- 강사 지인이나 이런건데 또 결제금액이 있는 것도 있고
- 쿠폰 중 실결제가가 0원인걸 하자니 1+1이 있고

In [193]:
len(df['coupon_title'].unique()), df['coupon_title'].unique()

(495,
 array([nan, '[10% 할인] 시크릿코드 실무 완성편', '[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰',
        '[웰컴쿠폰] 올인원 패키지 1만원 할인 쿠폰', '[웰컴쿠폰] 온라인 완주반 5만원 할인 쿠폰',
        '?즉시할인 쿠폰 코드 : spring5% ?', '[럭키룰렛] 올인원패키지',
        '[웰컴쿠폰] 올인원 패키지 3만원 할인 쿠폰', '타이포그래피 기수강생 할인 쿠폰',
        '[할인쿠폰] 파이낸스 카테고리 올인원 패키지 30% 할인 (금융 아카데미 제외)', '[럭키룰렛] 완주반',
        '[10% 할인] 프론트엔드 실무 완성편',
        '한 번에 끝내는 Java/Sring 웹 개발 마스터 초격차 패키지 무료 쿠폰 (-2/28)',
        '수강생 50만명 돌파 기념 감사제 쿠폰', '올인원 패키지 : 타이포그래피와 편집디자인 강의 수강권',
        '[무료수강권] 강의 검수용_이호상 PM님', '패캐머_온라인 쿠폰 !',
        '[무료수강권] 파이낸스 카테고리 올인원 패키지_대체투자 자산운용 실무 영상 오픈 일정 변경',
        '한 번에 끝내는 반복 업무 자동화 초격차 패키지 사용 가능 쿠폰', 'crm 검수용 쿠폰 aop',
        '[쿠폰재발급]수강생 50만명 돌파 기념 감사제 쿠폰', '[재발급] 페이백 쿠폰',
        '[무료수강권] 강의검수용_강사님 쿠폰_박혜린 강사님', '신해동 발행 쿠폰', '[무료수강권] 강사님 제공용 RED',
        '[무료수강권] 강사님 제공용', '[할인쿠폰] 영상/디자인 카테고리 올인원 패키지 5만원 할인_영상 오픈 일정 변경',
        '[무료수강권] 내부직원 검수용 RED',
        '[무료수강권] 올인원 패키지 : 인스타를 성장시킨 50명의 성공 방정식 강의검수용_강사님 쿠폰', '올인원패키지',
        '[무료수강권] 내부직원 강의 검수용',


In [147]:
df.to_csv('./모니터조.csv', index=False)   #  지금까지한 csv

In [279]:
df.to_csv('./task5다함께.csv', index=False)

In [204]:
df = pd.read_csv('./전처리다되었죠.csv')
df

,Unnamed: 0,id,customer_id,course_id,type,state,course_title,category_title,format,completed_at,transaction_amount,coupon_title,coupon_discount_amount,sale_price,pg,method,subcategory_title,marketing_start_at
0,0,970634,504760,201435,ORDER,CANCELLED,올인원 패키지 : 알고리즘 기술면접 완전 정복,프로그래밍,올인원,2021-03-28 15:00:11,90500,미적용,0,110500,COMM,VBANK,개발자 커리어,0
1,1,970650,432862,203178,ORDER,CANCELLED,초격차 패키지 : 한번에 끝내는 영상 촬영과 편집,영상/3D,올인원,2021-03-28 15:16:53,96000,미적용,0,116000,COMM,CARD,영상 편집,2020-12-02 00:00:00
2,2,970657,72297,204246,PAYMENT,COMPLETED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,[10% 할인] 시크릿코드 실무 완성편,19090,190900,COMM,CARD,프로그래밍,2021-03-04 00:00:00
3,3,970656,72297,204246,ORDER,CANCELLED,시크릿코드 : 프론트엔드 실무 완성편,프로그래밍,스쿨 온라인,2021-03-28 15:21:43,171810,미적용,0,190900,COMM,CARD,프로그래밍,2021-03-04 00:00:00
4,4,970658,478028,201797,PAYMENT,COMPLETED,올인원 패키지 : 직장인 필수 스킬 3종 세트 MAX,업무 생산성,올인원,2021-03-28 15:21:54,97500,[웰컴쿠폰] 올인원 패키지 2만원 할인 쿠폰,20000,117500,COMM,CARD,PPT/보고서,2019-11-14 00:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
101813,101813,1364995,640697,205365,PAYMENT,CANCELLED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:10:42,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000,116000,INICIS,TRANS,영상 편집,2021-06-15 00:00:00
101814,101814,1364998,670896,204159,PAYMENT,CANCELLED,부동산 디벨로퍼 아카데미 : 인허가,부동산/금융,올인원,2021-09-25 14:16:33,189000,미적용,0,189000,INICIS,CARD,부동산 개발,2021-02-10 00:00:00
101815,101815,1364996,640697,205365,PAYMENT,COMPLETED,초격차 패키지 : 한 번에 끝내는 유튜브 크리에이터 되기,영상/3D,올인원,2021-09-25 14:17:41,96000,[WELCOME] 온라인 강의 2만원 할인쿠폰,20000,116000,INICIS,VBANK,영상 편집,2021-06-15 00:00:00
101816,101816,1365005,670896,204159,PAYMENT,CANCELLED,부동산 디벨로퍼 아카데미 : 인허가,부동산/금융,올인원,2021-09-25 14:19:56,189000,미적용,0,189000,INICIS,CARD,부동산 개발,2021-02-10 00:00:00


In [184]:
df.isnull().sum()

Unnamed: 0                0
id                        0
customer_id               0
course_id                 0
type                      0
state                     0
course_title              0
category_title            0
format                    0
completed_at              0
transaction_amount        0
coupon_title              0
coupon_discount_amount    0
sale_price                0
pg                        0
method                    0
subcategory_title         0
marketing_start_at        0
dtype: int64

In [207]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101818 entries, 0 to 101817
Data columns (total 18 columns):
 #   Column                  Non-Null Count   Dtype 
---  ------                  --------------   ----- 
 0   Unnamed: 0              101818 non-null  int64 
 1   id                      101818 non-null  int64 
 2   customer_id             101818 non-null  int64 
 3   course_id               101818 non-null  int64 
 4   type                    101818 non-null  object
 5   state                   101818 non-null  object
 6   course_title            101818 non-null  object
 7   category_title          101818 non-null  object
 8   format                  101818 non-null  object
 9   completed_at            101818 non-null  object
 10  transaction_amount      101818 non-null  int64 
 11  coupon_title            101818 non-null  object
 12  coupon_discount_amount  101818 non-null  int64 
 13  sale_price              101818 non-null  int64 
 14  pg                      101818 non-n

In [205]:
df = df.astype({'marketing_start_at':'datetime64'})
df

ParserError: day is out of range for month: 0

In [ ]:
df = df.astype({'course_id':'float64'})
df

In [191]:
df[df['coupon_title'].str.contains('테스트')].value_counts().sum()

27

In [209]:
df.to_csv('./2022-06-29_bab_ppc.csv')